In [ ]:
pip install ipywidgets


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets

# Function to model Susceptible, Infected, and Recovered populations using different recovery rates for vaccinated and unvaccinated
def flu_model(infection_spread_rate, vaccinated_recovery_rate, unvaccinated_recovery_rate, vaccination_rate, peak_infection_day, total_days, population_size):
    # Simulating over a user-defined number of days with 100 points
    days = np.linspace(0, total_days, 100)

    # Split the population into vaccinated and unvaccinated groups based on vaccination rate
    vaccinated_population = population_size * vaccination_rate
    unvaccinated_population = population_size * (1 - vaccination_rate)

    # Recovered Population: Sigmoid function for vaccinated and unvaccinated people
    # Vaccinated population recovers faster
    recovered_vaccinated = (1 / (1 + np.exp(-vaccinated_recovery_rate * (days - total_days / 2)))) * vaccinated_population
    # Unvaccinated population recovers slower
    recovered_unvaccinated = (1 / (1 + np.exp(-unvaccinated_recovery_rate * (days - total_days / 2)))) * unvaccinated_population
    # Total recovered population is the sum of both vaccinated and unvaccinated recovered individuals
    recovered_population = recovered_vaccinated + recovered_unvaccinated

    # Infected Population: Gaussian (bell-shaped curve)
    # This models the infected population, peaking at 'peak_infection_day', scaled by the total population.
    infected_population = (np.exp(-infection_spread_rate * (days - peak_infection_day)**2)) * population_size

    # Susceptible Population: Remaining population that has not been infected or recovered
    susceptible_population = population_size - (infected_population + recovered_population)

    # Ensure no negative values for susceptible population due to floating-point errors
    susceptible_population[susceptible_population < 0] = 0

    # Plotting the results
    plt.figure(figsize=(10, 6))

    # Plot the susceptible population (Healthy, not yet infected)
    plt.plot(days, susceptible_population, label='Susceptible Population', color='orange')

    # Plot the infected population (Currently infected individuals)
    plt.plot(days, infected_population, label='Infected Population', color='red')

    # Plot the recovered population (People who have recovered)
    plt.plot(days, recovered_population, label='Recovered Population (Total)', color='green')

    # Plot the recovered vaccinated population
    plt.plot(days, recovered_vaccinated, '--', label='Recovered Vaccinated Population', color='blue')

    # Plot the recovered unvaccinated population
    plt.plot(days, recovered_unvaccinated, '--', label='Recovered Unvaccinated Population', color='purple')

    # Adding labels, title, and legend for better understanding
    plt.title(f'Flu Model: Susceptible, Infected, and Recovered Populations Over Time (Population: {population_size})')
    plt.xlabel('Days')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()

# Interact function with sliders for parameters, allowing interactive control of the simulation
interact(flu_model,
         infection_spread_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Infection Spread Rate'),
         vaccinated_recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.5, description='Vaccinated Recovery Rate'),
         unvaccinated_recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Unvaccinated Recovery Rate'),
         vaccination_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Vaccination Rate'),
         peak_infection_day=FloatSlider(min=1, max=20, step=1, value=5, description='Infection Peak Day'),
         total_days=IntSlider(min=10, max=60, step=1, value=30, description='Total Days'),
         population_size=IntSlider(min=100, max=1000000, step=1000, value=10000, description='Population Size'));


interactive(children=(FloatSlider(value=0.2, description='Infection Spread Rate', max=1.0, min=0.01, step=0.01…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets

# Function to model Susceptible, Infected, and Recovered populations using different recovery rates for vaccinated and unvaccinated
def flu_model(infection_spread_rate, vaccinated_recovery_rate, unvaccinated_recovery_rate, vaccination_rate, peak_infection_day, total_days, population_size):
    # Simulating over a user-defined number of days with 100 points
    days = np.linspace(0, total_days, 100)

    # Split the population into vaccinated and unvaccinated groups based on vaccination rate
    vaccinated_population = population_size * vaccination_rate
    unvaccinated_population = population_size * (1 - vaccination_rate)

    # Recovered Population: Sigmoid function for vaccinated and unvaccinated people
    # Vaccinated population recovers faster
    recovered_vaccinated = (1 / (1 + np.exp(-vaccinated_recovery_rate * (days - total_days / 2)))) * vaccinated_population
    # Unvaccinated population recovers slower
    recovered_unvaccinated = (1 / (1 + np.exp(-unvaccinated_recovery_rate * (days - total_days / 2)))) * unvaccinated_population
    # Total recovered population is the sum of both vaccinated and unvaccinated recovered individuals
    recovered_population = recovered_vaccinated + recovered_unvaccinated

    # Infected Population: Gaussian (bell-shaped curve)
    # This models the infected population, peaking at 'peak_infection_day', scaled by the total population.
    infected_population = (np.exp(-infection_spread_rate * (days - peak_infection_day)**2)) * population_size

    # Susceptible Population: Remaining population that has not been infected or recovered
    susceptible_population = population_size - (infected_population + recovered_population)

    # Ensure no negative values for susceptible population due to floating-point errors
    susceptible_population[susceptible_population < 0] = 0

    # Plotting the results

    # Figure 1: Susceptible, Infected, and Total Recovered Populations
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)  # First plot for SIR curves

    # Plot the susceptible population (Healthy, not yet infected)
    plt.plot(days, susceptible_population, label='Susceptible Population', color='orange')

    # Plot the infected population (Currently infected individuals)
    plt.plot(days, infected_population, label='Infected Population', color='red')

    # Plot the recovered population (People who have recovered)
    plt.plot(days, recovered_population, label='Total Recovered Population', color='green')

    # Adding labels, title, and legend for better understanding
    plt.title(f'Flu Model: Susceptible, Infected, and Recovered Populations Over Time')
    plt.xlabel('Days')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Figure 2: Recovered Vaccinated, Recovered Unvaccinated, and Total Recovered Populations
    plt.subplot(2, 1, 2)  # Second plot for vaccinated/unvaccinated curves

    # Plot the recovered vaccinated population
    plt.plot(days, recovered_vaccinated, '--', label='Recovered Vaccinated Population', color='blue')

    # Plot the recovered unvaccinated population
    plt.plot(days, recovered_unvaccinated, '--', label='Recovered Unvaccinated Population', color='purple')

    # Plot the total recovered population again for comparison
    plt.plot(days, recovered_population, label='Total Recovered Population', color='green')

    # Adding labels, title, and legend for the second graph
    plt.title(f'Recovered Populations: Vaccinated vs Unvaccinated')
    plt.xlabel('Days')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Adjust layout and show both plots
    plt.tight_layout()
    plt.show()

# Interact function with sliders for parameters, allowing interactive control of the simulation
interact(flu_model,
         infection_spread_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Infection Spread Rate'),
         vaccinated_recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.5, description='Vaccinated Recovery Rate'),
         unvaccinated_recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Unvaccinated Recovery Rate'),
         vaccination_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Vaccination Rate'),
         peak_infection_day=FloatSlider(min=1, max=20, step=1, value=5, description='Infection Peak Day'),
         total_days=IntSlider(min=10, max=60, step=1, value=30, description='Total Days'),
         population_size=IntSlider(min=100, max=1000000, step=1000, value=10000, description='Population Size'));


interactive(children=(FloatSlider(value=0.2, description='Infection Spread Rate', max=1.0, min=0.01, step=0.01…